# Solución y Tests para la Tarea 3
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo de ElGamal y firmas de Schnorr sobre curvas elípticas.

## Funciones auxiliares
Primero necesitamos un test de primalidad, para lo cual usamos lo mismo que para la pregunta 1 de la Tarea 2.

In [1]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [2]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    return _extended_euclid_base(b, a)

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [3]:
def _is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Una primera clase y sus elementos
Como un ejemplo de la forma en la cual debe ser implementado el un grupo en esta tarea, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente otra clase.

In [4]:
class ZpStar:
    
    def __init__(self, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform an element into a string
            def __str__(self):
                return str(self.value)
                
        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora usted debe completar las definiciones de las clases para enviar/recibir mensajes encriptados con ElGamal y generar/verificar firmas de Schnorr.

In [5]:
import hashlib

class SecretKeyHolder:
    def __init__(self, group, generator, subgroup_order):
        # Is the order of the generator correct? For this we check that
        # 1. The subgroup order is prime; and
        # 2. The generator to the power of subgroup_order is 1.
        if not _is_probably_prime(subgroup_order):
            raise Exception(f"subgroup_order={subgroup_order} is not a prime number")
        if not generator ** subgroup_order == group.get_identity():
            raise Exception(
                f"order={subgroup_order} is not the order of the subgroup generated by {generator}"
            )
            
        # The secret key is simply a scalar
        self.secret_key = random.randint(1, subgroup_order - 1)
        
        # The public key must contain the group, the generator,
        # the order of the generator, and the generator to the
        # power secret_key
        self.public_key = (
            group,
            generator,
            subgroup_order,
            generator ** self.secret_key
        )

    def get_public_key(self):
        return self.public_key
        
    def decrypt(self, ciphertext):
        # Returns decryption of ciphertext
        inverse = ciphertext[1] ** (self.public_key[2] - self.secret_key) 
        return (ciphertext[0] * inverse)
    
    def schnorr_signature(self, message):
        # Returns a Schnorr's signature of message
        hash = hashlib.sha256()
        k = random.randint(1, self.public_key[2] - 1)
        hash.update((str(self.public_key[1] ** k) + str(message)).encode())
        v = int.from_bytes(hash.digest(), 'big') % self.public_key[2]
        s = (k + v*self.secret_key) % self.public_key[2]
        return (v, s)

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `SecretKeyHolder` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `subgroup_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `subgroup_order` va a ser un número natural mayor o igual a 1).

**Importante:** En la generación de firmas de Schnorr se debe calcular el hash de un elemento del grupo concatenado con el mensaje, que también es un elemento del grupo. En esta concatenación estamos suponiendo una forma de transformar los elementos de `ZpStar` a string directa (dada por `str(n)`), pero para otros grupos podría no ser tan directa. Para evitar problemas usaremos lo mismo en cualquier grupo: dados dos elementos `g1` y `g2`, calcule el hash usando `hash.update((str(g1) + str(g2)).encode())`.

Además, el algoritmo utilizado para calcular una firma de Schnorr ``(v, s)`` debe ser el visto en clases, pero considerando que tanto ``v`` como ``s`` son calculados en módulo ``subgroup_order`` para reducir el tamaño de las firmas. Al implementar este paso, es importante que piense por qué las firmas de Schnorr son correctas si se utiliza módulo ``subgroup_order``.

In [6]:
class PublicKeyHolder:
    def __init__(self, pubkey):
        self.group = pubkey[0]
        self.generator = pubkey[1]
        self.subgroup_order = pubkey[2]
        self.pubkey = pubkey[3]

    def encrypt(self, message):
        # Definition of the ephemeral key to be used in the encryption
        my_exponent = random.randint(1, self.subgroup_order - 1)
        one_time_pubkey = self.generator ** my_exponent
        
        # In ElGamal the ciphertext is a pair of elements
        return (message * (self.pubkey ** my_exponent), one_time_pubkey)     
    
    def verify_schnorr_signature(self, message, signature):
        # Verify whether signature is a valid Schnorr's signature of message
        hash = hashlib.sha256()
        alpha = (self.generator ** signature[1]) * (self.pubkey ** (self.subgroup_order - signature[0]))
        hash.update((str(alpha) + str(message)).encode())
        return signature[0] == int.from_bytes(hash.digest(), 'big') % self.subgroup_order

Recuerde que estas clases están definidas para cualquier grupo, y por lo tanto se espera que su implementación funcione con una interfaz genérica para interactuar con estos objetos. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para el grupo Z<sub>643</sub><sup>\*</sup>.

In [7]:
if __name__ == "__main__":
    p, generator_value, order = 643, 4, 107
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = group.get_element(203)
    print(f"Plaintext:      {plaintext}")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:     ({ciphertext[0]}, {ciphertext[1]})")
    
    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text: {dec}")

Plaintext:      203
Ciphertext:     (399, 576)
Decrypted text: 203


Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar algo como lo siguiente:
```
Plaintext:      203
Ciphertext:     (449, 257)
Decrypted text: 203
```
Tanto el primer como el último mensaje deben ser `203`, mientras que el segundo mensaje debe ser un par ordenado que corresponde al cifrado de `203` utilizando la clave pública. 

Nótese que en este caso `203` es el mensaje a enviar, el cual es definido como un elemento del grupo Z<sub>643</sub><sup>\*</sup> a través de la línea `plaintext = group.get_element(203)`.

Como un segundo ejemplo considere un grupo Z<sub>p</sub><sup>\*</sup> que es usado en la práctica.

In [8]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    generator = group.get_element(generator)

    receiver = SecretKeyHolder(group, generator, order)

    sender = PublicKeyHolder(receiver.get_public_key())

    message = 989833749383746435764298374556465473646485709287354827346928387431239586091238465

    plaintext = group.get_element(message)
    print(f"Plaintext:      {plaintext}\n")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:     ({ciphertext[0]}, {ciphertext[1]})\n")
    
    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text: {dec}")

Plaintext:      989833749383746435764298374556465473646485709287354827346928387431239586091238465

Ciphertext:     (4702886023077716193929532516859914106407501416656952425687622033565131504120675071402945632717937447892324276672957650629710776704719307587359188001124983280838891999949072852735521003610187648947185321383318243126009505323896854825882085392555451852644347020474865382102474292885470518618782828337517406076947773212302447453508871449455295647110523170569216309401947881527954533756152703589486491744530310556701897579408542166263098761860181108513663164665065343803692702729604969558361918144044419029638918751398828920219189111328155365823025171814066770139072980578537978123327380962654912719601358036547651438938, 21786620478544233073672859992821173751155219954776996153372001157777611661845206709741895037228933386315311897197643407682996389059261257891258699666571871795940717042200639100427574155110116628811190489554704503134187227865641670667630747707918541707817656037826745

Una vez que haya completado las definiciones de las clases para enviar y recibir mensajes con ElGamal, el código anterior debe mostrar algo como lo siguiente:
```
Plaintext:      989833749383746435764298374556465473646485709287354827346928387431239586091238465

Ciphertext:     (12376884202351939903515713464996733144644281513601681814813650412361361208778683022094965067228766934463805643383190553315565394247262895451277187899288623335288739133516616819018361699994629372303953927513852709278368473229775734325410987018918442455262717729670812326044720888835987774805143709888133129436286772774526517354960473646500123257288428082565144986755834648946817089526639493836569724235553121709682730393321341887668400450269730784499268017736826342565277776968083316150294121864253119591642007349033841244564578781028780010400170685432154596024039131415179029541241552290271412615501222047264499223273, 6466456597689351504413352349798923193349043154550983607719992052401774657229783635049631669571499770269158624398195546512526454848710962660417444361415378989148102243475434306795803169529379533622509516043890117736986637976980922255390507455007232195711025170073429545247660618209319059004478689824567122562497582173750836424936778057977406668897430061205273036576316750107476234812452293957840366104827679547340186818966101512720408959562425181915979056950171766638028889178696162078219687184767975883719677585388469260886372538783912550708780834478915196082191110596023658708234500715066304717128037475087245132390)

Decrypted text: 989833749383746435764298374556465473646485709287354827346928387431239586091238465
```

En los siguientes ejemplos utilizamos las funciones para construir y verificar firmas de Schnorr. En primer lugar utilizamos el grupo Z<sub>643</sub><sup>\*</sup>.

In [9]:
if __name__ == "__main__":
    p, generator_value, order = 643, 4, 107
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    signer = SecretKeyHolder(group, generator, order)
    verifier = PublicKeyHolder(signer.get_public_key())

    message = group.get_element(413)
    signature = signer.schnorr_signature(message)

    print("Message:      ", str(message))
    print("Signature:    ", str(signature))
    print("Verification: ", verifier.verify_schnorr_signature(message, signature))

Message:       413
Signature:     (104, 102)
Verification:  True


En segundo lugar consideramos el grupo Z<sub>p</sub><sup>\*</sup> mencionado anteriormente, y que es utilizado en la práctica.

In [10]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator_value = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227
    
    group = ZpStar(p)
    generator = group.get_element(generator_value)

    signer = SecretKeyHolder(group, generator, order)
    verifier = PublicKeyHolder(signer.get_public_key())

    message_1 = group.get_element(98983374938374643576429837455646547364648570928735482734692838743)
    signature_1 = signer.schnorr_signature(message_1)
    
    message_2 = group.get_element(43563885929883747494886799876766827676119919203874473389748384984)
    signature_2 = signer.schnorr_signature(message_2)

    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))


Message:       98983374938374643576429837455646547364648570928735482734692838743
Signature:     (51638364831343177632024895458674993368163736786737918751414217764902363181170, 10694301622427305512254307047086157505870295183661355275019925131836777237985)
Verification:  True

Message:       43563885929883747494886799876766827676119919203874473389748384984
Signature:     (4282636982932898204436231485984608689927970607569893270261025625761362038215, 30937412485431649124212698070009314112016801432839827658635396039573761131980)
Verification:  True

Message:       98983374938374643576429837455646547364648570928735482734692838743
Signature:     (4282636982932898204436231485984608689927970607569893270261025625761362038215, 30937412485431649124212698070009314112016801432839827658635396039573761131980)
Verification:  False

Message:       43563885929883747494886799876766827676119919203874473389748384984
Signature:     (516383648313431776320248954586749933681637367867379187514142177649023631811

## Utilizando curvas elípticas
En esta segunda parte de la tarea, usted debe utilizar firmas de Schnorr y encriptación de ElGamal tal como antes, pero esta vez sobre grupos definidos por curvas elípticas. En particular, debe completar la siguiente definición de la clase `EllipticCurve` considerando la definición de curvas elípticas dada en la ecuación (9.2) del la sección 9.3.4 del libro:

Jonathan Katz y Yehuda Lindell. Introduction to Modern Cryptography. Chapman and Hall/CRC,
tercera edición, 2020.

In [11]:
class EllipticCurve:
    def __init__(self, A, B, p):
        if not _is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        elif A < 0 or A > p-1:
            raise Exception(f"A={A} is not in the range 0,...,{p-1}")
        elif B < 0 or B > p-1:
            raise Exception(f"B={B} is not in the range 0,...,{p-1}")
        elif (4*pow(A,3) + 27*pow(B,2)) % p == 0:
            raise Exception(f"The determinant is equal to 0 modulo p={p}")             
        class Element:
            def __init__(self, x, y = None):
                if y is None:
                    if not x == 'N':
                        raise Exception(f"x={x} must take value N")
                    self.x = x
                elif x < 0 or x > p-1:
                    raise Exception(f"x={x} is not in the range 0,...,{p-1}")
                elif y < 0 or y > p-1:
                    raise Exception(f"y={y} is not in the range 0,...,{p-1}")
                elif not (pow(y,2) - (pow(x,3) + A*x + B)) % p == 0: 
                    raise Exception(f"Point x={x}, y={y} is outside the elliptic curve")
                else:
                    self.x = x
                    self.y = y

            def __eq__(self, other_element):
                if hasattr(self,'y'):
                    if hasattr(other_element,'y'):
                        return (self.x == other_element.x and self.y == other_element.y)
                    return False
                else:
                    if not hasattr(other_element, 'y'):
                        return (self.x == other_element.x)
                    return False

            def __mul__(self, other_element):
                if (self.x == 'N'):
                    return other_element
                elif (other_element.x == 'N'):
                    return self
                elif not self.x == other_element.x:
                    s = ((self.y - other_element.y)*pow(self.x - other_element.x, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                elif not self.y == other_element.y:
                    return Element('N')
                elif not self.y == 0:
                    s = ((3*pow(self.x, 2) + A)*pow(2*self.y, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                return Element('N')
                    
            def __pow__(self, exponent):
                if exponent == 0:
                    return Element('N')
                elif exponent == 1:
                    return self
                else:
                    result = Element('N')
                    pot = self
                    while exponent > 0:
                        if exponent % 2 == 1:
                            result = pot * result
                        exponent = exponent // 2
                        pot = pot * pot
                return result            
            
            
            def __str__(self):
                if hasattr(self,'y'):
                    return f"({self.x}, {self.y})"
                return "N"
                    
        self.element_class = Element
                
    def get_identity(self):
        return self.element_class('N')
                    
    def get_element(self, x, y):
        return self.element_class(x,y)

En esta definición de `EllipticCurve`, dado un número primo `p`, cada punto sobre la curva es un par ordenado `(x,y)` con `x` e `y` en el conjunto `{0, ..., p-1}`, excepto por el neutro del grupo que un elemento especial que no necesita notación de par ordenado (ver el libro de Katz & Lindell para una explicación de esto). Por esto el constructor de la clase `EllipticCurve` recibe dos argumentos para representar un par ordenado, y también considera el caso en que `y` no esté definido porque se está utilizando el elemento neutro.

**Importante:** Para evitar problemas, dado un elemento `g = (x, y)` del grupo, diremos que su interpretación como string es literalmente `(x, y)` (notar el espacio después de la coma). Es decir, la interpretación será como string se debe calcular con algo como `f"({self.x}, {self.y})"`.

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `EllipticCurve` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `p` no es un número primo, entonces se debe generar una excepción.

Su definición de la clase `EllipticCurve` va a ser utilizada por la implementación del protocolo ElGamal de la misma forma que para la clase `ZpStar`. Por ejemplo, en el siguiente código se utiliza el protocolo ElGamal para la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256).

In [12]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    receiver = SecretKeyHolder(group, g, q)

    sender = PublicKeyHolder(receiver.get_public_key())

    message_x = 3649244856384847635638847363849074342342433643773
    message_y = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    
    plaintext = group.get_element(message_x, message_y)
    print(f"Plaintext:       {str(plaintext)}\n")
    
    ciphertext = sender.encrypt(plaintext)
    print(f"Ciphertext:      [{ciphertext[0]}, {ciphertext[1]}]\n")

    dec = receiver.decrypt(ciphertext)
    print(f"Decrypted text:  {str(dec)}")

Plaintext:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)

Ciphertext:      [(71155008274268767298844962600864165437346808871046609953179777353420531935305, 46519033332751334114685608926522584173082952680427983299383272897291079540819), (21983676071086044821845415647293773750734645977752587675470908292904788017016, 58710620416250573297262450086211678289740499779443339776296841599987932903031)]

Decrypted text:  (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)


Dadas las clases implementadas en la primer parte de la tarea, el código anterior debe mostrar algo como lo siguiente:

```
Plaintext:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)

Ciphertext:      [(113996131010303204014009892935779309769658129295209181632143587339087143687314, 26962689901361466823054095068011324081132818595053733951227392568180562298562), (54755798469491832606228455763246989103441843832178270077860596483712932816750, 80388391817873044711837096504683074525746519031165376005382087646062080905529)]

Decrypted text:  (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
```
Nótese que en este caso

```
(3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
```

es el mensaje a enviar, el cual es definido como un elemento del grupo a través de la línea `plaintext = group.get_element(message_x, message_y)`

Finalmente, utilizamos las funciones para construir y verificar firmas de Schnorr para la curva elíptica considerada anteriormente.

In [13]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)

    signer = SecretKeyHolder(group, g, q)
    pub_key = signer.get_public_key()
    verifier = PublicKeyHolder(pub_key)

    print("\nThe public key (g**x) is:")
    print(pub_key[3])

    message_x_1 = 3649244856384847635638847363849074342342433643773
    message_y_1 = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    message_1 = group.get_element(message_x_1, message_y_1)
    signature_1 = signer.schnorr_signature(message_1)

    message_x_2 = 59447290591372491095936616477776244661201105999102239672215969253558897392491
    message_y_2 = 113121750122093533018561227344023152845279298590622316429489010324710260843069
    message_2 = group.get_element(message_x_2, message_y_2)
    signature_2 = signer.schnorr_signature(message_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))


The public key (g**x) is:
(42636645101578620175040185615232389092399926761263416875316259780062566878525, 76834725829297073538713765881285182291734820566794833842419757400132167906153)

Message:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
Signature:     (15783657984736575665138613860680023277936415577443108425921899164255803615840, 98629767334415251556037621554291852313094035894406815320380867367828181813729)
Verification:  True

Message:       (59447290591372491095936616477776244661201105999102239672215969253558897392491, 113121750122093533018561227344023152845279298590622316429489010324710260843069)
Signature:     (53894491665989495284360387009491836268034946807731082916200390936915111182801, 95566781103748205162771813082410868666655804889966593511707653573309950517947)
Verification:  True

Message:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909

## Verificación
Aunque las Firmas de Schnorr son aleatorizadas (es decir, dos firmas del mismo mensaje muy probablemente van a ser distintas), su verificación es obviamente determinista.

A continuación se utilizan ciertos valores obtenidos del output de una ejecución del código anterior, el cual incluye mensajes, firmas y verificaciones. Si su clase de curvas elípticas fue programada correctamente, el siguiente código debería correr sin problemas y generar un output posible en el mismo formato que el que se muestra arriba.

In [14]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    q = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    pubkey_x = 103192619728557224015619336422717788072328875409700539750964537777199132907664
    pubkey_y = 93298080101102371782236605662083759019495876114486733420689273227397978704898
    pubkey_group_element = group.get_element(pubkey_x, pubkey_y)

    pubkey = (group, g, q, pubkey_group_element)
    verifier = PublicKeyHolder(pubkey)

    print("\nThe public key (g**x) is:")
    print(pubkey[3])

    message_x_1 = 3649244856384847635638847363849074342342433643773
    message_y_1 = 36810392828448194526040058211987909976903679270241111391326603075746535787758
    message_1 = group.get_element(message_x_1, message_y_1)

    signature_x_1 = 49880133444030498374826058973351206690403473262559368006905951025443593632411
    signature_y_1 = 103065885058305515152187888940408087212139311086906277275899754836127292868590
    signature_1 = (signature_x_1, signature_y_1)

    message_x_2 = 59447290591372491095936616477776244661201105999102239672215969253558897392491
    message_y_2 = 113121750122093533018561227344023152845279298590622316429489010324710260843069
    message_2 = group.get_element(message_x_2, message_y_2)

    signature_x_2 = 106108015439441123124532815622331817639891188646096647525674670688491309960170
    signature_y_2 = 52823047958769667531168620359625033126342114852292082076787832317629658210545
    signature_2 = (signature_x_2, signature_y_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_1))
    assert verifier.verify_schnorr_signature(message_1, signature_1)
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_2))
    assert verifier.verify_schnorr_signature(message_2, signature_2)
    
    print("\nMessage:      ", str(message_1))
    print("Signature:    ", str(signature_2))
    print("Verification: ", verifier.verify_schnorr_signature(message_1, signature_2))
    assert not verifier.verify_schnorr_signature(message_1, signature_2)
    
    print("\nMessage:      ", str(message_2))
    print("Signature:    ", str(signature_1))
    print("Verification: ", verifier.verify_schnorr_signature(message_2, signature_1))
    assert not verifier.verify_schnorr_signature(message_2, signature_1)


The public key (g**x) is:
(103192619728557224015619336422717788072328875409700539750964537777199132907664, 93298080101102371782236605662083759019495876114486733420689273227397978704898)

Message:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987909976903679270241111391326603075746535787758)
Signature:     (49880133444030498374826058973351206690403473262559368006905951025443593632411, 103065885058305515152187888940408087212139311086906277275899754836127292868590)
Verification:  True

Message:       (59447290591372491095936616477776244661201105999102239672215969253558897392491, 113121750122093533018561227344023152845279298590622316429489010324710260843069)
Signature:     (106108015439441123124532815622331817639891188646096647525674670688491309960170, 52823047958769667531168620359625033126342114852292082076787832317629658210545)
Verification:  True

Message:       (3649244856384847635638847363849074342342433643773, 36810392828448194526040058211987

## Tests

Para verificar si su tarea funciona bien se realizarán los siguientes tests.

### Tests para grupo ZpStar

In [15]:
p = 5809605995369958062791915965639201402176612226902900533702900882779736177890990861472094774477339581147373410185646378328043729800750470098210924487866935059164371588168047540943981644516632755067501626434556398193186628990071248660819361205119793693985433297036118232914410171876807536457391277857011849897410207519105333355801121109356897459426271845471397952675959440793493071628394122780510124618488232602464649876850458861245784240929258426287699705312584509625419513463605155428017165714465363094021609290561084025893662561222573202082865797821865270991145082200656978177192827024538990239969175546190770645685893438011714430426409338676314743571154537142031573004276428701433036381801705308659830751190352946025482059931306571004727362479688415574702596946457770284148435989129632853918392117997472632693078113129886487399347796982772784615865232621289656944284216824611318709764535152507354116344703769998514148343807
generator_value = 2
order = 2904802997684979031395957982819600701088306113451450266851450441389868088945495430736047387238669790573686705092823189164021864900375235049105462243933467529582185794084023770471990822258316377533750813217278199096593314495035624330409680602559896846992716648518059116457205085938403768228695638928505924948705103759552666677900560554678448729713135922735698976337979720396746535814197061390255062309244116301232324938425229430622892120464629213143849852656292254812709756731802577714008582857232681547010804645280542012946831280611286601041432898910932635495572541100328489088596413512269495119984587773095385322842946719005857215213204669338157371785577268571015786502138214350716518190900852654329915375595176473012741029965653285502363681239844207787351298473228885142074217994564816426959196058998736316346539056564943243699673898491386392307932616310644828472142108412305659354882267576253677058172351884999257074171903

group = ZpStar(p)
generator = group.get_element(generator_value)


def test_order_identity():
    return generator**order == group.get_identity()
    
    
def _generate_message():
    message = random.randint(0, p-1)
    return group.get_element(message)


def test_distinct_encrypt():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = _generate_message()
    ciphertext_1 = sender.encrypt(plaintext)
    ciphertext_2 = sender.encrypt(plaintext)
    return not ciphertext_1 == ciphertext_2


def test_encrypt_decrypt():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = _generate_message()
    ciphertext = sender.encrypt(plaintext)
    return plaintext == receiver.decrypt(ciphertext)   


def test_signatures():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())
    
    message_1 = _generate_message()
    message_2 = _generate_message()
    signature_1 = receiver.schnorr_signature(message_1)
    signature_2 = receiver.schnorr_signature(message_2)

    return (sender.verify_schnorr_signature(message_1, signature_1) and 
            not sender.verify_schnorr_signature(message_1, signature_2) and
            not sender.verify_schnorr_signature(message_2, signature_1) and
            sender.verify_schnorr_signature(message_2, signature_2))


def test_fixed_signatures():
    receiver = SecretKeyHolder(group, generator, order)
    receiver.secret_key = 1785520638739734098737227118474597531102242928882915888751831012354271655596608478516084768891616866272919536393716756715709634645379999215356600122406266241614918793699548013731152040030979902196249171745089878229922119373253915355288950590645953179092200524183746804546641507126484917492618746469989952096037778344640889709862006939142477889218589798321257519699596743848766151326269880647255587090355192102417815069501899089121254297103134726458284350732410681593794406176368453357797838100668081595997191186145664187663505003772326536755484591814534230775999474357073329566104102275733031102819494172275687711585589058666046048334459576511559220421770899264450771742066464064738571346523737976797476624175962299137988075769406452936769745174347055456784111664270936389554001770601114872199421984397598050984058066238473435615670968234753526275197324670026642461860988614474914473382242676004920602092635829001706883008235
    receiver.public_key = (group, generator, order, generator ** receiver.secret_key)
    sender = PublicKeyHolder(receiver.get_public_key())
     
    message_1 = 87487239487657402390845345445537575535245362636359843474874982675847795654983644345465672
    message_2 = 342356423654768673454784758475655675968607960898908028328130823908055738944739483282349483044
    
    signature_1 = (106288277326165761843569262860185019571592487210527596339624790318909383314368,
                   705331229827156372532716646028278791035648771280402685530377812506975594377994388060595257062082315072197502849736319041746776206338796124522581421903331500662466266644110178654988618937298446649036106393063699485097615895502928978967533479130931403143048792624440377353613984168075225602125329038308289116196292258498989314665870797030843561816780010537028684922665124952041995646627289580018119418865428218905297158775309976976628511339191119398405098400604467328837690085501145393127388321404870322716450676683355345609329750795682179399300322635358700424089828277711037496347121143477647625269126492667384157448203556415186810891952443835349021359445442796247832189722327106721872352790934056167071688303409565530530504762053439150851431160121594690021378842938149557732272575231762520240372445988921768171755112555134554080693675847417558305444296659139928112102218316471046238751471239276586509800922280594884900896426)
    signature_2 = (33992219382347423363264494600880972257015240671442667033836250581058592382374,
                   2191531925221427037563980935674253872934392969055268522391724289931390770511732493215951549908463942083898880211424219573236000549761772142347603245086288645198472139101753612263625441116035657043583938582972131480916242393385227941661726088450607020657703431182110244348061372386416647641732169000136031278223010304875575466807873923565266452695161868910989868949900134886299412641746589083442640490155429366401579972277923366889908451007546868063777097954622052687314853013523763108538222583364468239595214746415568938731139777688117825105186524736218900595646422485742736963809762604478989518283469530058540713494084561129900121180867097368993270014125604223346423169379478115947426607651141133399752263236817114696654730269668159436922013755247567460597016886844382465467484217097443720415221665357057982794138577378378626176682551682815398877005432911550470751549366114071588315661160297431196652812075990936939994945615)
    
    return (sender.verify_schnorr_signature(message_1, signature_1) and 
            not sender.verify_schnorr_signature(message_1, signature_2) and
            not sender.verify_schnorr_signature(message_2, signature_1) and
            sender.verify_schnorr_signature(message_2, signature_2))

Se espera que el valor de todos los siguientes tests sea `True`.

In [16]:
print(test_order_identity())
print(test_distinct_encrypt())
print(test_encrypt_decrypt())
print(test_signatures())
print(test_fixed_signatures())

True
True
True
True
True


### Test para grupo EllipticCurve

In [17]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
order = 115792089210356248762697446949407573529996955224135760342422259061068512044369

group = EllipticCurve(A, B, p)
generator = group.get_element(g_x, g_y)


def _is_sqrt(a, p):
    if a % p == 0:
        return True
    elif pow(a, (p-1)//2, p) == 1:
        return True
    return False


def _sqrt_prime(a, p):
    if a % p == 0:
        return 0
    elif not _is_sqrt(a, p):
        raise Exception(f"{a} does not have a square root in module {p}")
    gamma = 1
    while _is_sqrt(gamma, p):
        gamma = random.randint(2, p-1)
    t = 0
    while (p - 1) % pow(2,t+1) == 0:
        t += 1
    s = (p - 1)//pow(2, t)
    K = []
    i = 2
    while i <= t:
        value = pow(a, pow(2, t-i)*s, p)
        for j in range(0, len(K)):
            value = (value * pow(gamma, pow(2, t-i+j+1)*s*K[j], p)) % p
        if value == 1:
            K.append(0)
        else:
            K.append(1)
        i += 1
    raiz = pow(a, (s+1)//2, p)
    for j in range(0, len(K)):
        raiz = (raiz * pow(gamma, K[j]*s*pow(2, j), p)) % p
    return(raiz)


def _generate_message():
    message_x = random.randint(0, p-1)
    while not _is_sqrt((pow(message_x,3) + A*message_x + B) % p, p):
        message_x = random.randint(0, p-1)
    message_y = _sqrt_prime((pow(message_x,3) + A*message_x + B) % p, p)
    return group.get_element(message_x, message_y)


def test_order_identity():
    return generator**order == group.get_identity()


def test_order_not_prime():
    order = 1157920892103562487626974469494075735999695522413576034422259061068512044369
    try:
        receiver = SecretKeyHolder(group, generator, order)
        return False
    except:
        return True

    
def test_incorrect_point():
    message_x = 8584884342863516615406165511406323351118960803751730059471035603840445150962
    message_y = 98909719646589390956366800466550261365057590657394075725444138176993443443851
    
    try:
        messaje = group.get_element(message_x, message_y)
        return False
    except:
        return True

    
def test_distinct_encrypt():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = _generate_message()
    ciphertext_1 = sender.encrypt(plaintext)
    ciphertext_2 = sender.encrypt(plaintext)
    return not ciphertext_1 == ciphertext_2


def test_encrypt_decrypt():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())

    plaintext = _generate_message()
    ciphertext = sender.encrypt(plaintext)
    return plaintext == receiver.decrypt(ciphertext)   


def test_signatures():
    receiver = SecretKeyHolder(group, generator, order)
    sender = PublicKeyHolder(receiver.get_public_key())
    
    message_1 = _generate_message()
    message_2 = _generate_message()
    signature_1 = receiver.schnorr_signature(message_1)
    signature_2 = receiver.schnorr_signature(message_2)

    return (sender.verify_schnorr_signature(message_1, signature_1) and 
            not sender.verify_schnorr_signature(message_1, signature_2) and
            not sender.verify_schnorr_signature(message_2, signature_1) and
            sender.verify_schnorr_signature(message_2, signature_2))


def test_fixed_signatures():
    receiver = SecretKeyHolder(group, generator, order)
    receiver.secret_key = 70356421495684926937876466808719853649892785673198267186953811513642829862421
    receiver.public_key = (group, generator, order, generator ** receiver.secret_key)
    sender = PublicKeyHolder(receiver.get_public_key())
     
    message_x_1 = 76649984980555424634884504804763288158516615548348238199529385011237269059655 
    message_y_1 = 21104261356503354155088464026807043243254738187722405474904399933682671184127
    message_1 = group.get_element(message_x_1, message_y_1)    
        
    message_x_2 = 37151822412953665499932049518405876648757871402573110402850250804141190423032
    message_y_2 = 22536693846859314636150386227033125968516523016443065800020773700626955747569
    message_2 = group.get_element(message_x_2, message_y_2)
    
    signature_1 = (23338523114811393118653330746019059116363514042563338511174461701416051934615, 
                   115358941752066948439067147778163947175873045835240836064474917173919855484482)
    signature_2 = (49535133630393081769555281609884359135035858765993874328335282315374790161662, 
                   7127942716994367277277029904519466391708184664980418935383123510308948422376)
    
    return (sender.verify_schnorr_signature(message_1, signature_1) and 
            not sender.verify_schnorr_signature(message_1, signature_2) and
            not sender.verify_schnorr_signature(message_2, signature_1) and
            sender.verify_schnorr_signature(message_2, signature_2))

Como en el caso anterior, se espera que el valor de todos los siguientes tests sea `True`.

In [18]:
print(test_order_identity())
print(test_order_not_prime())
print(test_incorrect_point())
print(test_distinct_encrypt())
print(test_encrypt_decrypt())
print(test_signatures())
print(test_fixed_signatures())

True
True
True
True
True
True
True
